# Costa Rican Household Poverty Level Prediction
## Exploratory Data Analysis and Predictive Modeling


Developed by **jhonnatan.torres.suarez@gmail.com**

**Notes:** 
- English is not my native language, so my apologies in advance for any typo or grammar mistake
- The model used in this kernel was not optimized using GridSearch or RandomSearch
- LB Score **0.393**
________________________________________________________________________________________________________________________________________________________________________________________________________________

First of all, the usual/custom libraries are imported

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import os
print(os.listdir("../input"))

Setting the max columns options of Pandas to 150 (in order to visualize the tables with all the features / variables)

In [ ]:
pd.set_option('display.max_columns', 150)

Reading the data

In [ ]:
tr = pd.read_csv('../input/train.csv')
ts = pd.read_csv('../input/test.csv')

Checking the structure, variables, rows and columns

In [ ]:
tr.head()

In [ ]:
ts.head()

The training dataset is smaller than the submission dataset

In [ ]:
print(tr.shape)
print(ts.shape)

In [ ]:
print(tr.columns.values)
print('\n')
print(ts.columns.values)

Including a mock/dummy value in order to concatenate the training and submission datasets

In [ ]:
ts['Target']=5

In [ ]:
df = pd.concat([tr,ts],axis=0)

In [ ]:
df.describe(include='all')

Checking the blank values

In [ ]:
df.isna().sum()

In [ ]:
#plt.figure(figsize=(18,6))
#sns.countplot(x='dependency',data=tr)
df['Vol']=1
pd.pivot_table(df,values='Vol',columns='dependency',aggfunc='sum')

In [ ]:
pd.pivot_table(df,values='Vol',columns='edjefe',aggfunc='sum')

In [ ]:
pd.pivot_table(df,values='Vol',columns='edjefa',aggfunc='sum')

It seems like some numbers were included instead of **yes**, the next step is to replace these numbers with **yes**

In [ ]:
df['dependency_f']=np.where(df['dependency']!='yes','no','yes')
df['dependency_f'].value_counts()

In [ ]:
df['edjefe_f']=np.where(df['edjefe']!='yes','no','yes')
df['edjefe_f'].value_counts()

In [ ]:
df['edjefa_f']=np.where(df['edjefa']!='yes','no','yes')
df['edjefa_f'].value_counts()

Removing the columns with numbers and categories and another colums with a high volume of zeros (0)

In [ ]:
df.drop(columns=['dependency','edjefa','edjefe','v2a1','v18q1','rez_esc'],inplace=True)

In [ ]:
sns.set_context('notebook')
sns.set_style('darkgrid')

Next steps is to creates a couple of histograms of the **SQBmeaned** and the **meaneduc**

In [ ]:
print(df['SQBmeaned'].describe(include='all'))
df['SQBmeaned'].plot.hist(bins=30)
plt.grid(alpha=0.25)

In [ ]:
print(df['meaneduc'].describe(include='all'))
df['meaneduc'].plot.hist(bins=30)
plt.grid(alpha=0.25)

Replacing the null values with the mean/average

In [ ]:
df['SQBmeaned'].fillna(value=np.mean(df['SQBmeaned']),inplace=True)

In [ ]:
df['meaneduc'].fillna(value=np.mean(df['meaneduc']),inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df.describe(include='all')

In [ ]:
print(df['edjefa_f'].value_counts())
print(df['elimbasu4'].value_counts())

Mostly of the variables included in the dataset have been binarized, even, there are 2 columns, one for males and another one for females, hence some columns can be removed

In [ ]:
df.drop(columns=['Id','r4h3','r4m3','r4t3','paredother','pisoother','techootro','energcocinar4','elimbasu4','elimbasu5','elimbasu6','epared3','etecho3','eviv3','male','estadocivil7',
                'parentesco12','idhogar','instlevel9','tipovivi5','lugar6','area2','Vol'],inplace=True)

Replacing **yes** and **no** for **1** and **0**

In [ ]:
df['edjefe_f']=np.where(df['edjefe_f']=='yes',1,0)
df['edjefa_f']=np.where(df['edjefa_f']=='yes',1,0)
df['dependency_f']=np.where(df['dependency_f']=='yes',1,0)

Scaling the numeric features

In [ ]:
sm=df.describe(include='all')
sm.iloc[7,:].sort_values(ascending=False).head(30).index

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
dfs=MinMaxScaler().fit_transform(df[['agesq', 'SQBage', 'SQBmeaned', 'SQBedjefe', 'SQBescolari',
       'SQBhogar_total', 'SQBovercrowding', 'SQBhogar_nin', 'age',
       'SQBdependency', 'meaneduc', 'escolari', 'tamviv', 'rooms',
       'hogar_total', 'hhsize', 'overcrowding', 'tamhog', 'qmobilephone',
       'r4t2', 'hogar_nin', 'r4t1', 'bedrooms', 'hogar_adul', 'r4h2', 'r4m2',
       'r4m1', 'r4h1', 'hogar_mayor']])

In [ ]:
dfsc = pd.DataFrame(data=dfs,columns=['agesq', 'SQBage', 'SQBmeaned', 'SQBedjefe', 'SQBescolari',
       'SQBhogar_total', 'SQBovercrowding', 'SQBhogar_nin', 'age',
       'SQBdependency', 'meaneduc', 'escolari', 'tamviv', 'rooms',
       'hogar_total', 'hhsize', 'overcrowding', 'tamhog', 'qmobilephone',
       'r4t2', 'hogar_nin', 'r4t1', 'bedrooms', 'hogar_adul', 'r4h2', 'r4m2',
       'r4m1', 'r4h1', 'hogar_mayor'])
dfsc.describe()

A boxplot of the numeric features

In [ ]:
g = dfsc.plot.box(figsize=(20,4))
for item in g.get_xticklabels():
    item.set_rotation(60)

Removing the original numeric features

In [ ]:
df.drop(columns=['agesq', 'SQBage', 'SQBmeaned', 'SQBedjefe', 'SQBescolari',
       'SQBhogar_total', 'SQBovercrowding', 'SQBhogar_nin', 'age',
       'SQBdependency', 'meaneduc', 'escolari', 'tamviv', 'rooms',
       'hogar_total', 'hhsize', 'overcrowding', 'tamhog', 'qmobilephone',
       'r4t2', 'hogar_nin', 'r4t1', 'bedrooms', 'hogar_adul', 'r4h2', 'r4m2',
       'r4m1', 'r4h1', 'hogar_mayor'],inplace=True)

Including the scaled numeric features into the original dataframe

In [ ]:
dfsc.index=df.index
print(df.shape)
print(dfsc.shape)
ndf=pd.concat([df,dfsc],axis=1)
print(ndf.shape)
ndf.head()
ndf['Target'].value_counts()

Filtering by the competition dataset

In [ ]:
dw = ndf[ndf['Target']!=5]
print(dw.shape)
dw['Target'].value_counts()

Correlation heatmap of the dataset features

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(dw.corr(),cmap='coolwarm')

# Machine Learning Stage

Importing the models/algorithms to be used

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import scikitplot as skplt

Decision Tree (used to filter the most important features)

In [ ]:
rfc = DecisionTreeClassifier(criterion='entropy',class_weight='balanced',random_state=1234)

In [ ]:
rfc.fit(dw.drop(columns='Target'),dw['Target'])

In [ ]:
fi = pd.DataFrame(data=rfc.feature_importances_,index=dw.drop(columns='Target').columns,columns=['Importance'])
fi.sort_values(by='Importance',ascending=False).plot.bar(figsize=(20,4))
plt.grid(alpha=0.25)

Top 15 features

In [ ]:
fi.sort_values(by='Importance',ascending=False).head(15).index

In [ ]:
feats =fi.sort_values(by='Importance',ascending=False).head(15).index

In [ ]:
from sklearn.model_selection import train_test_split

Boxplot of features

In [ ]:
g = dw[feats].plot.box(figsize=(18,4))
for item in g.get_xticklabels():
    item.set_rotation(60)

80% - 20% ratio was used

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dw[feats],dw['Target'], test_size=0.20, random_state=1234)

Simple bagging classifier

In [ ]:
model = BaggingClassifier(n_estimators=2500,random_state=1234,warm_start=False,verbose=1,oob_score=True)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

In [ ]:
prds = model.predict(X_test)

In [ ]:
print(classification_report(y_test,prds))
skplt.metrics.plot_confusion_matrix(y_test,prds,normalize=True)

In [ ]:
#cvs= cross_val_score(model,dw[feats],dw['Target'],cv=5,scoring='f1_macro')

In [ ]:
#print(cvs)
#print(np.mean(cvs))
#print(np.median(cvs))

In [ ]:
ds = ndf[ndf['Target']==5]
ds.head()
ds.drop(columns='Target',inplace=True)
print(ds.shape)

In [ ]:
bagprds = model.predict(ds[feats])

In [ ]:
pdf = pd.DataFrame(data=bagprds,columns=['Target'])

In [ ]:
pdf.index = ts.index
gsub = pd.concat([ts['Id'],pdf],axis=1)
gsub.head()
sns.countplot(gsub['Target'])
plt.grid()

In [ ]:
gsub.to_csv('bag_model_submit.csv',index =False,index_label=False)

In [ ]:
sns.countplot(ndf['Target'])

In [ ]:
k = dw.drop('Target',axis=1).plot.box(figsize=(40,10))
for item in k.get_xticklabels():
    item.set_rotation(60)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

A XGB Classifier was implemented

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgbc = XGBClassifier(n_estimators=7500,random_state=1234,max_depth=7)

In [ ]:
xgbc.fit(X_train,y_train)

In [ ]:
print(classification_report(y_test,xgbc.predict(X_test)))
skplt.metrics.plot_confusion_matrix(y_test,xgbc.predict(X_test),normalize=True)

A voting classifier was implemeted using the bagging and XGB models

In [ ]:
vc = VotingClassifier(estimators=[('bag', model), ('xgb', xgbc)], voting='hard')

In [ ]:
vc.fit(X_train,y_train)

In [ ]:
print(classification_report(y_test,vc.predict(X_test)))
skplt.metrics.plot_confusion_matrix(y_test,vc.predict(X_test),normalize=True)

Cross validation to assess the model (voting) performance 

In [ ]:
cvvs= cross_val_score(vc,dw[feats],dw['Target'],cv=5,scoring='f1_macro')
print(cvvs)
print("Average Cross Validation Score: ",np.mean(cvvs))
print("Median Cross Validation Score: ",np.median(cvvs))

Predicting using the submission data and creating the submission file

In [ ]:
bagprdss = vc.predict(ds[feats])
pdfs = pd.DataFrame(data=bagprdss,columns=['Target'])

pdfs.index = ts.index
gsubs = pd.concat([ts['Id'],pdfs],axis=1)
gsubs.head()
sns.countplot(gsubs['Target'])
plt.grid()

gsubs.to_csv('voter_submit.csv',index =False,index_label=False)